<a href="https://colab.research.google.com/github/mmmovania/2dsmoke/blob/master/Week12/CUDA_DynamicParallelism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

/usr/local
  File: cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 23h/35d	Inode: 82          Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-04-06 09:14:03.915133239 +0000
Modify: 2022-04-06 09:14:03.806133512 +0000
Change: 2022-04-06 09:14:03.806133512 +0000
 Birth: -
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dw832sw9
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dw832sw9
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=f71b3faee3e26cc74b7bb1bd82b4b330847708a24d1765a099ac25ced979bfee
  Stored in directory: /tmp/pip-ephem-wheel-cache-gsi_xjaj/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
directory /usr/local/src already exists
Out bin /usr/local/result.out


In [12]:
%%cu

//this example wont work on Colab GPU since the required compute capability for 
//kernel launch from another kernel is compute capability 3.5

#include <stdio.h>
const int N = 100;//33 * 1024;
const int threadsPerBlock = 32;// 256;

#define imin(a,b) (a<b?a:b)

const int blocksPerGrid =  imin( 32, (N+threadsPerBlock-1) / threadsPerBlock );

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

__global__ void kernel( float *a, float *b, int N ) 
{
	int i = threadIdx.x + blockDim.x*blockIdx.x;
	if(i < N)
	{
			for(int j=i; j< min(i+5, N); j++)
			{
					b[i] += a[j];
			}
	}
}

__global__ void kernel_child(int start, int end, float *a, float *b) 
{
	int j = start + threadIdx.x + blockDim.x*blockIdx.x;
	if(j < end)
	{
			b[j] += a[j];			
	}
}

__global__ void kernel_parent( float *a, float *b, int end ) 
{
	int i = threadIdx.x + blockDim.x*blockIdx.x;

	b[i] = a[i];	
	kernel_child<<<1, 32>>>(i, min(i+5, end), a, b);			 
}



int main() 
{ 
	float   *a = 0; 
  float   *b = 0;
	float   *c = 0;

	// Allocate Unified Memory -- accessible from CPU or GPU
	checkCudaErr(cudaMallocManaged(&a, N*sizeof(float)), "cudaMallocManaged1");
	checkCudaErr(cudaMallocManaged(&b, N*sizeof(float)), "cudaMallocManaged2"); 
	checkCudaErr(cudaMallocManaged(&c, N*sizeof(float)), "cudaMallocManaged3"); 

	// fill in the memory with data
	for (int i=0; i<N; i++) {
		a[i] = i+1; 
		b[i] = 0;
		c[i] = 0;
	} 

	kernel<<<blocksPerGrid,threadsPerBlock>>>( a, b, N );

	cudaDeviceSynchronize();

	kernel_parent<<<blocksPerGrid,threadsPerBlock>>>( a, b, N );
  
	for(int i=0; i<N;++i)
		printf("%f\n", b[i]);
	
	// free memory on the gpu side
	checkCudaErr( cudaFree( a ) , "cudaFree1");
	checkCudaErr( cudaFree( b ) , "cudaFree2"); 
	checkCudaErr( cudaFree( c ) , "cudaFree3"); 
	checkCudaErr( cudaDeviceReset(), "cudaDeviceReset");

	return 0;
}

/tmp/tmpk8633i3c/b77ace81-d6a5-4843-bd1c-c6a9444890a1.cu(46): error: calling a __global__ function("kernel_child") from a __global__ function("kernel_parent") is only allowed on the compute_35 architecture or above

1 error detected in the compilation of "/tmp/tmpxft_0000023a_00000000-8_b77ace81-d6a5-4843-bd1c-c6a9444890a1.cpp1.ii".

